# 실전크롤링 - 2. 네이버 지식인 크롤링
- 지식in 내 제목, 링크, 날짜, 카테고리, 답변수(숫자만) 크롤링
- 특이한 선택자 case (class명에 특수문자, 띄어쓰기가 포함된 경우)
    - 클래스명 사이에 띄어쓰기가 있는 경우 : 공백 기준 여러 클래스명이 결합한 것임
        - 각각 따로 찾아도 되고, 같이 사용하고 싶은 경우 공백을 '.'으로 연결
    - 클래스명에 특수문자가 있는 경우 : 해당 특수문자 앞에 '\\'로 escape 문법 적용
- 사용자입력 처리

## 1) 1페이지 첫번째 질문 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90' #삼성전자 검색

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

title = soup.select_one('._nclicks\\:kin\\.txt').text
link = soup.select_one('.basic1 a').attrs['href'] #바로 뒤에 오는 태그는 >, 뒤 아무곳이나 오는 태그는 ' '
ddate = soup.select_one('.txt_inline').text
category = soup.select_one('.txt_g1._nclicks\\:kin\\.cat2').text #방법1
category = soup.select_one('.txt_block > a:nth-of-type(2)').text #방법2
answer = soup.select_one('.hit').text.split(' ')[1] #숫자만 추출 #방법1



'14'

## 2) 1페이지 10개 질문 크롤링
- 크롤링은 '비로그인 상태'로 크롤링하기 때문에 로그인 상태와 크롤링 결과가 다를 수 있음

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90' #삼성전자 검색

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

tags = soup.select('.basic1 > li') #나무태그 찾기
for tag in tags:
    title = tag.select_one('._nclicks\\:kin\\.txt').text
    link = tag.select_one('.basic1 a').attrs['href']
    ddate = tag.select_one('.txt_inline').text
    category = tag.select_one('.txt_g1._nclicks\\:kin\\.cat2').text
    answer = tag.select_one('.hit').text.split(' ')[1]

    print(title, link, ddate, category, answer)
    print('--------------')
    

삼성전자 주식 https://kin.naver.com/qna/detail.naver?d1id=4&dirId=40102&docId=487572786&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.07.30. 주식, 증권 14
--------------
삼성전자 에어컨 청소 https://kin.naver.com/qna/detail.naver?d1id=5&dirId=50104&docId=487926458&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.11. 청소, 주방, 계절 가전 9
--------------
삼성전자 주가 전망 https://kin.naver.com/qna/detail.naver?d1id=4&dirId=40102&docId=429328715&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.07.30. 주식, 증권 4
--------------
삼성전자에 벤큐 모니터 파나요 https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1010401&docId=488053141&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.16. 모니터 4
--------------
삼성전자대리점에서 소비쿠폰을 사용할...  https://kin.naver.com/qna/detail.naver?d1id=5&dirId=50104&docId=487725447&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.08.06. 청소, 주방, 계절 가전 2
--------------
삼성전자 상간녀소송중인데요. 상간녀가 https://kin.naver.com/qna/detail.naver?d1id=6&dirId=60204&docId=302160869&qb=7IK87ISx7KCE7J6Q&enc=utf8 2025.05.27. 가족, 이혼 3
--------------
삼성전자서비스센터 USB 복구, 수리 가능? https://kin.naver.com/qna/detail.n

## 3) N페이지까지 크롤링(사용자 입력)

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

cnt = input("크롤링할 페이지 수를 입력해주세요.")
data = []

for i in range(1, int(cnt)+1):
    url = 'https://kin.naver.com/search/list.naver?query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90' + f'&page={str(i)}' #삼성전자 검색
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.select('.basic1 > li') #나무태그 찾기
    for tag in tags:
        title = tag.select_one('._nclicks\\:kin\\.txt').text
        link = tag.select_one('.basic1 a').attrs['href']
        ddate = tag.select_one('.txt_inline').text
        category = tag.select_one('.txt_g1._nclicks\\:kin\\.cat2').text
        answer = tag.select_one('.hit').text.split(' ')[1]

        data.append([title, link, ddate, category, int(answer)])

df = pd.DataFrame(data, columns=['title', 'link', 'ddate', 'category', 'answer'])
df.to_excel(f'C:/Users/qawse/세희/IT공부/크롤링_inflearn/naver_지식in_crawling.xlsx', index=None)       

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

input_keyword ='주식'
input_page='2'
for i in range(1, int(input_page)+1):
    response = requests.get(f"https://kin.naver.com/search/list.naver?query={quote(input_keyword)}&page={i}")
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.select('.basic1 > li') #나무태그 찾기

for tag in tags:
    title = tag.select_one('._nclicks\\:kin\\.txt').text
    link = tag.select_one('.basic1 a').attrs['href']
    ddate = tag.select_one('.txt_inline').text
    category = tag.select_one('.txt_g1._nclicks\\:kin\\.cat2').text
    answer = tag.select_one('.hit').text.split(' ')[1]